<a href="https://colab.research.google.com/github/piyush-bhandari-commits/self-learn-repo/blob/master/Text_Classification_H2O_AutoML_and_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 129.3MB 70kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.1-py2.py3-none-any.whl size=129358602 sha256=16f7763fdf04e5b0a58df2b766c1a0ea31cb97f5289d0d4e17f180cb8a95b2c7
  Stored in directory: /root/.cache/pip/wheels/33/ac/52/165c35d747abdb629c3c9fb7e087f360c662d8cb58824caed8
Successfully built h2o


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import seaborn as sns
from pprint import pprint
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8hytvwd8
  JVM stdout: /tmp/tmp8hytvwd8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8hytvwd8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_q0riht
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
# Fetching the dataset from the data.gov website
url = 'https://files.consumerfinance.gov/ccdb/complaints.csv.zip'
actual_data = pd.read_csv(url, compression='zip')

In [7]:
# Pre-preocessing the actual dataset
print ('Shape of Actual Data: {}'.format(actual_data.shape))

Shape of Actual Data: (1729208, 18)


In [8]:
actual_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [9]:
raw_dataset = actual_data[[actual_data.columns[1], actual_data.columns[5], actual_data.columns[7]]].rename(columns={'Consumer complaint narrative': 'Complaint'})
clean_raw_dataset = raw_dataset.dropna()
print('The dataset has {} rows and {} columns'.format(clean_raw_dataset.shape[0], clean_raw_dataset.shape[1]))
clean_raw_dataset.head()

The dataset has 585595 rows and 3 columns


,Product,Complaint,Company
0,Debt collection,transworld systems inc. \nis trying to collect...,TRANSWORLD SYSTEMS INC
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...","Diversified Consultants, Inc."
15,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...","Paypal Holdings, Inc"
16,Mortgage,This complaint dates back to last XX/XX/XXXX d...,NATIONSTAR MORTGAGE


In [10]:
clean_raw_dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    197287
Debt collection                                                                 120384
Mortgage                                                                         68055
Credit card or prepaid card                                                      41380
Credit reporting                                                                 31588
Student loan                                                                     26756
Checking or savings account                                                      23940
Credit card                                                                      18838
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                               10158
Vehicle loan or lease                                                            10056
Consumer Loan                              

In [11]:
dataset = clean_raw_dataset.sample(n=100000)
dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    33632
Debt collection                                                                 20603
Mortgage                                                                        11553
Credit card or prepaid card                                                      7203
Credit reporting                                                                 5334
Student loan                                                                     4492
Checking or savings account                                                      4047
Credit card                                                                      3257
Bank account or service                                                          2515
Vehicle loan or lease                                                            1782
Money transfer, virtual currency, or money service                               1739
Consumer Loan                                         

In [12]:
label_count = dataset['Product'].value_counts().to_dict()
for label, count in label_count.items():
    if count < 3000 or count > 21000:
        dataset = dataset.drop(dataset[dataset.Product.values == label].index)

In [13]:
dataset.Product.value_counts()

Debt collection                20603
Mortgage                       11553
Credit card or prepaid card     7203
Credit reporting                5334
Student loan                    4492
Checking or savings account     4047
Credit card                     3257
Name: Product, dtype: int64

In [14]:
dataset.Company.value_counts()

CITIBANK, N.A.                           2685
Navient Solutions, LLC.                  2321
BANK OF AMERICA, NATIONAL ASSOCIATION    2287
JPMORGAN CHASE & CO.                     2244
WELLS FARGO & COMPANY                    2109
                                         ... 
Townstone Financial Inc                     1
KAHRS LAW OFFICES, PA                       1
LCS FINANCIAL SERVICES CORPORATION          1
Mirand Response Systems, Inc.               1
National Student Debt Advisory Corp.        1
Name: Company, Length: 2434, dtype: int64

In [15]:
mapping = {}
for value in np.arange(len(np.unique(dataset.Product.values))):
    mapping[np.unique(dataset.Product.values)[value]] = value 

In [16]:
mapping

{'Checking or savings account': 0,
 'Credit card': 1,
 'Credit card or prepaid card': 2,
 'Credit reporting': 3,
 'Debt collection': 4,
 'Mortgage': 5,
 'Student loan': 6}

In [17]:
dataset['Target'] = dataset.Product.map(mapping)
dataset.head()

,Product,Complaint,Company,Target
1689728,Debt collection,I called Alco Collections and asked about the ...,"Alco Collections, Inc.",4
1428277,Debt collection,"Calls from "" Harris and Harris '' a debt colle...","Harris & Harris, Ltd.",4
1728772,Student loan,I called last year in XXXX to try to lower my ...,"Navient Solutions, LLC.",6
507499,Checking or savings account,This is a small town bank who is abusing their...,BB&T CORPORATION,0
538408,Checking or savings account,Between XXXX XXXX and XXXX I visited a merchan...,PNC Bank N.A.,0


In [18]:
# Splitting data into train and test set
train, test = train_test_split(dataset, test_size=0.20, random_state=42)

In [19]:
print ('Shape of Train Data : {}'.format(train.shape))
print ('Shape of Test Data : {}'.format(test.shape))

Shape of Train Data : (45191, 4)
Shape of Test Data : (11298, 4)


In [20]:
pprint (train['Complaint'].values[10])

('I contacted the credit bureau to inform them of XXXX hard inquiries on my '
 'credit report that are not accurate. I requested of them to be removed. In '
 "response I was told it 's nothing they can do about them. They ca n't remove "
 'the hard inquires. It states that if something appears on my report that is '
 'not accurate I have the right to request to have it disputed.')


In [21]:
pprint (train['Complaint'].values[28])

('On XXXX XXXX, I received an email from Experian. \n'
 'Subject : Notification : Your XXXX Score has changed! \n'
 'I clicked the button that read : " Check your XXXX Score Now \'\' I landed '
 'on a webpage with XXXX buttons. " See all XXXX scores \'\' or " See XXXX '
 "score '' There was absolutely no indication I was placing an order. \n"
 'No information that my credit card would be charged. \n'
 'No visible dollar amounts. \n'
 'No please review and confirm your order page. \n'
 'No email confirmation that I had placed an order. \n'
 'I have been scammed by Experian as they charged my credit card {$39.00} '
 'without my authorization. I have sent Experian an email requesting a refund '
 'just prior to filling out this form. CFPB, Please see what you can do to '
 'facilitate a prompt refund of {$39.00}.')


In [22]:
pprint (train['Complaint'].values[32])

('XX/XX/XXXXMy name is XXXX XXXX XXXX. I have a house here in XXXX Hawaii, We '
 'bought the house on XX/XX/XXXX when I was married then we refinance it '
 'XX/XX/XXXX and then me and my ex got divorce. after we got divorce I been '
 'fight to save the house its because its is under my name since XX/XX/XXXX '
 "which I did n't know till three months before my ex filed the divorce paper. "
 'By XX/XX/XXXX I found out that he never paid for three months and I tried to '
 "pay the behind payments all the way till XX/XX/XXXX. Then that 's when the "
 'XXXX XXXX has been harassing me calls and soliciting collecting debts. By '
 'XX/XX/XXXX I send them a hardship letter stating that due to my divorce I '
 'can only afford {$1200.00} a month plus I applied for modification with my '
 'hardship letter, they said I wont be qualified for modification because of '
 'my income is not enough, and which they accept that payments up until '
 'XX/XX/XXXX. They sent me a letter stating they cant accept

In [23]:
lemma = WordNetLemmatizer()

porter = nltk.PorterStemmer()

def clean_text(text):
    no_url_text = re.sub(r"http\S+", "", text)
    
    no_punct_text = re.sub("[^a-zA-Z#]", " ", no_url_text)
    
    no_num_text = re.sub("\d+{4-6}", " ", no_punct_text)

    no_encrypt_text = " ".join([word.strip('Xx/') for word in no_num_text.split()])
                               
    no_small_text = " ".join([word for word in no_encrypt_text.split() if len(word)>2])
    
    no_stop_text = " ".join([word for word in no_small_text.split() if word not in stopwords])
    
    # lemma_text = " ".join([lemma.lemmatize(word) for word in no_stop_text.split()])
    # stem_text = " ".join([porter.stem(word) for word in no_stop_text.split()])
    
    tokens = re.split('\W+', no_stop_text)
    return tokens

train['Complaint Tokens'] = train['Complaint'].apply(lambda x: clean_text(x.lower()))
train['Complaint Sentence'] = train['Complaint Tokens'].apply(lambda x: " ".join(x))
train.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Product,Complaint,Company,Target,Complaint Tokens,Complaint Sentence
1639326,Debt collection,Disputed with the company on XX/XX/2018. Compa...,Santander Consumer USA Holdings Inc.,4,"[disputed, company, company, received, dispute...",disputed company company received disputes sho...
517273,Credit card or prepaid card,Dear Blackhawk Network ( BHN ) : After finding...,Blackhawk Network Holdings Inc.,2,"[dear, blackhawk, network, bhn, finding, gift,...",dear blackhawk network bhn finding gift card e...
1315971,Mortgage,Ocwen continuously applies my monthly payments...,Ocwen Financial Corporation,5,"[ocwen, continuously, applies, monthly, paymen...",ocwen continuously applies monthly payments pr...
1443533,Credit card,I made a reservation at a resort for ten night...,JPMORGAN CHASE & CO.,1,"[made, reservation, resort, ten, nights, lagoo...",made reservation resort ten nights lagoon view...
1347692,Mortgage,There has been fraud and robo-signing going on...,Ocwen Financial Corporation,5,"[fraud, robo, signing, going, companies, handl...",fraud robo signing going companies handle note...


In [24]:
x_train = train['Complaint Sentence']
y_train = train['Target']

x_test = test['Complaint']
y_test = test['Target']
x_train.head()

1639326    disputed company company received disputes sho...
517273     dear blackhawk network bhn finding gift card e...
1315971    ocwen continuously applies monthly payments pr...
1443533    made reservation resort ten nights lagoon view...
1347692    fraud robo signing going companies handle note...
Name: Complaint Sentence, dtype: object

In [25]:
pprint (train['Complaint Sentence'].values[32])

('name house hawaii bought house married refinance got divorce got divorce '
 'fight save house name since know till three months filed divorce paper found '
 'never paid three months tried pay behind payments way till harassing calls '
 'soliciting collecting debts send hardship letter stating due divorce afford '
 'month plus applied modification hardship letter said wont qualified '
 'modification income enough accept payments sent letter stating cant accept '
 'payments anymore applied mortgage ask payments went put payments mine fund '
 'since fighting struggling handling harassment makes sick went hired lawyers '
 'lawyers find help tried sell house fighting attempting foreclose house five '
 'years seek help form mainland thought could help found research mortgage '
 'found rebo signing records mortgage keep transferring servicer company ocwen '
 'consultant helping writer response letter filing response motion company '
 'represent schedule appear court want sign approval judge

In [26]:
pprint (train['Complaint Sentence'].values[28])

('received email experian subject notification score changed clicked button '
 'read check score landed webpage buttons see scores see score absolutely '
 'indication placing order information credit card would charged visible '
 'dollar amounts please review confirm order page email confirmation placed '
 'order scammed experian charged credit card without authorization sent '
 'experian email requesting refund prior filling form cfpb please see '
 'facilitate prompt refund')


In [27]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1,2))
x_train_vect = tfidf_vectorizer.fit_transform(x_train)
x_test_vect = tfidf_vectorizer.fit_transform(x_test)

In [28]:
x_train_vect.shape, x_test_vect.shape

((45191, 1000), (11298, 1000))

In [29]:
df_train = pd.DataFrame(x_train_vect.toarray(), columns=tfidf_vectorizer.get_feature_names())
df_train = pd.concat([df_train, train['Target'].reset_index(drop=True)], axis=1)

In [30]:
df_test = pd.DataFrame(x_test_vect.toarray(), columns=tfidf_vectorizer.get_feature_names())
df_test = pd.concat([df_test, test['Target'].reset_index(drop=True)], axis=1)

In [31]:
h2o_train_df = h2o.H2OFrame(df_train)
h2o_test_df = h2o.H2OFrame(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
h2o_train_df['Target'] = h2o_train_df['Target'].asfactor()
h2o_test_df['Target'] = h2o_test_df['Target'].asfactor()

In [39]:
auto_ml = H2OAutoML(max_models=5, seed=10, exclude_algos=["StackedEnsemble"], verbosity="info", nfolds=0, balance_classes=True, max_after_balance_size=0.30)

In [40]:
x = tfidf_vectorizer.get_feature_names()
y = 'Target'

In [ ]:
auto_ml.train(x=x, y=y, training_frame=h2o_train_df, validation_frame=h2o_test_df)

AutoML progress: |
06:13:47.754: Project: AutoML_20200828_61347754
06:13:47.754: Cross-validation disabled by user: no fold column nor nfolds > 1.
06:13:47.755: Setting stopping tolerance adaptively based on the training frame: 0.004704072691053999
06:13:47.755: Build control seed: 10
06:13:47.755: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
06:13:51.343: training frame: Frame key: automl_training_py_1_sid_a794    cols: 1001    rows: 40577  chunks: 52    size: 30822671  checksum: -7098203681158365336
06:13:51.345: validation frame: Frame key: py_2_sid_a794    cols: 1001    rows: 11298  chunks: 16    size: 17424514  checksum: 6932471759648991472
06:13:51.591: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_a794    cols: 1001    rows: 4614  chunks: 52    size: 7117986  checksum: -3735611203821453608
06:13:51.592: blending frame: NULL
06:13:51.592: respons

In [36]:
auto_ml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_2_AutoML_20200828_054348,0.247256,0.488526,0.387016,0.149781
XGBoost_1_AutoML_20200828_054348,0.261246,0.509706,0.39761,0.158094
XGBoost_3_AutoML_20200828_054348,0.274705,0.524691,0.406833,0.165513
DRF_1_AutoML_20200828_054348,0.35344,0.706268,0.490096,0.240194
GLM_1_AutoML_20200828_054348,0.500101,1.05272,0.622262,0.387209


In [37]:
auto_ml.leader.predict(h2o_test_df)

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1,p2,p3,p4,p5,p6
5,0.0187364,0.00945012,0.0379152,0.0510893,0.0688552,0.669768,0.144186
3,0.0192304,0.0196709,0.102508,0.717201,0.0142523,0.121085,0.0060519
5,0.0121394,0.00852944,0.0248098,0.0359794,0.0811236,0.511147,0.326272
5,0.00960032,0.00165659,0.000943493,0.000854323,0.0600331,0.905673,0.0212396
2,0.0121653,0.0559444,0.384711,0.0888645,0.143442,0.244189,0.0706842
6,0.0104107,0.00496395,0.0162916,0.0187729,0.199167,0.285384,0.46501
5,0.0294724,0.0100281,0.0717286,0.00587646,0.401933,0.473351,0.00761088
4,0.0139212,0.00646103,0.05082,0.00844737,0.513974,0.403442,0.00293438
5,0.0118726,0.00338828,0.0124685,0.244291,0.0724067,0.6349,0.0206729
5,0.0054169,0.011371,0.0174201,0.0135418,0.0242385,0.924956,0.00305576


In [38]:
auto_ml.leader.model_performance(h2o_test_df)


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.6323404692723518
RMSE: 0.7951983835951578
LogLoss: 2.3746309812399304
Mean Per-Class Error: 0.7996887869285441

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,Error,Rate
0,23.0,5.0,99.0,99.0,172.0,409.0,53.0,0.973256,837 / 860
1,38.0,0.0,32.0,36.0,96.0,399.0,22.0,1.000000,623 / 623
2,54.0,5.0,66.0,96.0,273.0,858.0,58.0,0.953191,"1,344 / 1,410"
3,87.0,1.0,48.0,177.0,362.0,346.0,40.0,0.833176,"884 / 1,061"
4,124.0,10.0,118.0,483.0,1330.0,1805.0,199.0,0.673138,"2,739 / 4,069"
5,22.0,8.0,66.0,200.0,293.0,1665.0,110.0,0.295685,"699 / 2,364"
6,13.0,0.0,15.0,90.0,142.0,532.0,119.0,0.869374,792 / 911
7,361.0,29.0,444.0,1181.0,2668.0,6014.0,601.0,0.700832,"7,918 / 11,298"



Top-7 Hit Ratios: 


,k,hit_ratio
0,1,0.299168
1,2,0.507435
2,3,0.650381
3,4,0.764029
4,5,0.860860
5,6,0.939547
6,7,1.000000
